# ETL notebook

In [3]:
pip install pandas sqlalchemy pymysql nbformat

  Using cached pandas-2.3.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached pymysql-1.1.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached fastjsonschema-2.21.2-py3-none-any.whl.metadata (2.3 kB)
Using cached pandas-2.3.3-cp310-cp310-win_amd64.whl (11.3 MB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/

In [6]:
import pandas as pd

csv_path = "sales.csv"

encodings_to_try = [
    "utf-8",
    "latin1",
    "cp1252",
    "iso-8859-1",
    "iso-8859-15",
    "windows-1250",
    "windows-1252",
    "utf-16",
    "utf-16le",
    "utf-16be",
]

df = None
last_error = None

for enc in encodings_to_try:
    try:
        df = pd.read_csv(csv_path, encoding=enc)
        print(f"Successfully read CSV with encoding: {enc}")
        break
    except Exception as e:
        last_error = e
        print(f"Failed with encoding: {enc}")

if df is None:
    print("\n❌ Could not read file with any encoding.")
    raise last_error

print("\nShape:", df.shape)
df.head()


Failed with encoding: utf-8
Successfully read CSV with encoding: latin1

Shape: (2823, 25)


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


## Extract

In [ ]:
# imports and file path
# Run this cell first. It only loads libraries and defines csv path.
import pandas as pd
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")

csv_path = "sales.csv"

In [ ]:
# read CSV safely (we already know latin1 works, but we try fallback)
# This cell prints encoding used, shape and column names exactly as they appear.
encodings = ['utf-8','latin1','cp1252','iso-8859-1']
df = None
used_enc = None
for enc in encodings:
    try:
        df = pd.read_csv(csv_path, encoding=enc)
        used_enc = enc
        break
    except Exception:
        continue

if df is None:
    raise RuntimeError("Could not read CSV with tried encodings. Upload file or change path.")

print("CSV loaded with encoding:", used_enc)
print("Shape:", df.shape)
print("Columns (exact):")
for i,c in enumerate(df.columns):
    print(i+1, "-", repr(c))


CSV loaded with encoding: latin1
Shape: (2823, 25)
Columns (exact):
1 - 'ORDERNUMBER'
2 - 'QUANTITYORDERED'
3 - 'PRICEEACH'
4 - 'ORDERLINENUMBER'
5 - 'SALES'
6 - 'ORDERDATE'
7 - 'STATUS'
8 - 'QTR_ID'
9 - 'MONTH_ID'
10 - 'YEAR_ID'
11 - 'PRODUCTLINE'
12 - 'MSRP'
13 - 'PRODUCTCODE'
14 - 'CUSTOMERNAME'
15 - 'PHONE'
16 - 'ADDRESSLINE1'
17 - 'ADDRESSLINE2'
18 - 'CITY'
19 - 'STATE'
20 - 'POSTALCODE'
21 - 'COUNTRY'
22 - 'TERRITORY'
23 - 'CONTACTLASTNAME'
24 - 'CONTACTFIRSTNAME'
25 - 'DEALSIZE'


In [9]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


## Transform

In [ ]:
# show dtypes and non-null counts to decide transformations
print(df.dtypes)
print("\nNon-null counts:")
print(df.notnull().sum())


ORDERNUMBER           int64
QUANTITYORDERED       int64
PRICEEACH           float64
ORDERLINENUMBER       int64
SALES               float64
ORDERDATE            object
STATUS               object
QTR_ID                int64
MONTH_ID              int64
YEAR_ID               int64
PRODUCTLINE          object
MSRP                  int64
PRODUCTCODE          object
CUSTOMERNAME         object
PHONE                object
ADDRESSLINE1         object
ADDRESSLINE2         object
CITY                 object
STATE                object
POSTALCODE           object
COUNTRY              object
TERRITORY            object
CONTACTLASTNAME      object
CONTACTFIRSTNAME     object
DEALSIZE             object
dtype: object

Non-null counts:
ORDERNUMBER         2823
QUANTITYORDERED     2823
PRICEEACH           2823
ORDERLINENUMBER     2823
SALES               2823
ORDERDATE           2823
STATUS              2823
QTR_ID              2823
MONTH_ID            2823
YEAR_ID             2823
PRODUCTLINE       

In [ ]:
# create normalized lowercase column names mapping for internal use
# We won't overwrite original names in the CSV; this is for coding convenience.
orig_cols = df.columns.tolist()
# create a mapping: uppercase original -> lower_snake for code consistency
col_map = {c: c.strip() for c in orig_cols}  # keep original names intact, but we will reference exact originals below
col_map  # inspect mapping (no change, this cell is just informative)


{'ORDERNUMBER': 'ORDERNUMBER',
 'QUANTITYORDERED': 'QUANTITYORDERED',
 'PRICEEACH': 'PRICEEACH',
 'ORDERLINENUMBER': 'ORDERLINENUMBER',
 'SALES': 'SALES',
 'ORDERDATE': 'ORDERDATE',
 'STATUS': 'STATUS',
 'QTR_ID': 'QTR_ID',
 'MONTH_ID': 'MONTH_ID',
 'YEAR_ID': 'YEAR_ID',
 'PRODUCTLINE': 'PRODUCTLINE',
 'MSRP': 'MSRP',
 'PRODUCTCODE': 'PRODUCTCODE',
 'CUSTOMERNAME': 'CUSTOMERNAME',
 'PHONE': 'PHONE',
 'ADDRESSLINE1': 'ADDRESSLINE1',
 'ADDRESSLINE2': 'ADDRESSLINE2',
 'CITY': 'CITY',
 'STATE': 'STATE',
 'POSTALCODE': 'POSTALCODE',
 'COUNTRY': 'COUNTRY',
 'TERRITORY': 'TERRITORY',
 'CONTACTLASTNAME': 'CONTACTLASTNAME',
 'CONTACTFIRSTNAME': 'CONTACTFIRSTNAME',
 'DEALSIZE': 'DEALSIZE'}

### Transform: drop duplicates

In [ ]:
# drop exact duplicate rows
before = len(df)
df = df.drop_duplicates().reset_index(drop=True)
after = len(df)
print("Dropped duplicates:", before-after, "rows removed. New shape:", df.shape)


Dropped duplicates: 0 rows removed. New shape: (2823, 25)


### Transform: parse order_date if present

In [ ]:
# parse ORDERDATE column to date

if 'ORDERDATE' in df.columns:
    df['ORDERDATE_parsed'] = pd.to_datetime(df['ORDERDATE'], errors='coerce')
    print("ORDERDATE_parsed dtype:", df['ORDERDATE_parsed'].dtype)
    display(df[['ORDERDATE','ORDERDATE_parsed']].head(5))
else:
    print("No ORDERDATE column found.")


ORDERDATE_parsed dtype: datetime64[ns]


,ORDERDATE,ORDERDATE_parsed
0,2/24/2003 0:00,2003-02-24
1,5/7/2003 0:00,2003-05-07
2,7/1/2003 0:00,2003-07-01
3,8/25/2003 0:00,2003-08-25
4,10/10/2003 0:00,2003-10-10


### Transform: Numeric conversions: QUANTITYORDERED, PRICEEACH, SALES

In [ ]:
# convert numeric columns; create safe copies
num_cols = ['QUANTITYORDERED','PRICEEACH','SALES','MSRP']
for c in num_cols:
    if c in df.columns:
        df[c + '_num'] = pd.to_numeric(df[c], errors='coerce')
        print(f"{c}: converted, nulls = {df[c + '_num'].isnull().sum()}")


QUANTITYORDERED: converted, nulls = 0
PRICEEACH: converted, nulls = 0
SALES: converted, nulls = 0
MSRP: converted, nulls = 0


In [ ]:
# ensure essential fields exist per business rules
# Required for a valid order line: ORDERNUMBER, PRODUCTCODE, QUANTITYORDERED
required = []
for c in ['ORDERNUMBER','PRODUCTCODE','QUANTITYORDERED']:
    if c in df.columns:
        required.append(c)

print("Required columns present:", required)
if required:
    before = len(df)
    df = df.dropna(subset=required).reset_index(drop=True)
    print("Dropped rows missing required:", before - len(df))
    print("Shape now:", df.shape)


Required columns present: ['ORDERNUMBER', 'PRODUCTCODE', 'QUANTITYORDERED']
Dropped rows missing required: 0
Shape now: (2823, 30)


In [ ]:
# trim whitespace from string columns commonly used as keys
for c in ['PRODUCTCODE','CUSTOMERNAME','COUNTRY','CITY','CONTACTFIRSTNAME','CONTACTLASTNAME']:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip()
print("Trim complete.")


Trim complete.


### Transform: Create normalized tables

In [ ]:
# Customers table
# create one customer row per CUSTOMERNAME (the dataset contains company names and contact names)
cust_cols = [c for c in [
    'CUSTOMERNAME','CONTACTFIRSTNAME','CONTACTLASTNAME','PHONE',
    'ADDRESSLINE1','ADDRESSLINE2','CITY','STATE','POSTALCODE','COUNTRY','TERRITORY'
] if c in df.columns]

customers = df[cust_cols].drop_duplicates().reset_index(drop=True).copy()

# Give a clean column name that will map to DB
customers = customers.rename(columns={
    'CUSTOMERNAME':'customer_name',
    'CONTACTFIRSTNAME':'contact_firstname',
    'CONTACTLASTNAME':'contact_lastname',
    'PHONE':'phone',
    'ADDRESSLINE1':'addressline1',
    'ADDRESSLINE2':'addressline2',
    'CITY':'city',
    'STATE':'state',
    'POSTALCODE':'postalcode',
    'COUNTRY':'country',
    'TERRITORY':'territory'
})
print("Customers rows:", customers.shape[0])
customers.head()


Customers rows: 92


,customer_name,contact_firstname,contact_lastname,phone,addressline1,addressline2,city,state,postalcode,country,territory
0,Land of Toys Inc.,Kwai,Yu,2125557818,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN
1,Reims Collectables,Paul,Henriot,26.47.1555,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA
2,Lyon Souveniers,Daniel,Da Cunha,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA
3,Toys4GrownUps.com,Julie,Young,6265557265,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN
4,Corporate Gift Ideas Co.,Julie,Brown,6505551386,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN


In [ ]:
# Products table
prod_cols = [c for c in ['PRODUCTCODE','PRODUCTLINE','MSRP'] if c in df.columns]
products = df[prod_cols].drop_duplicates().reset_index(drop=True).copy()
products = products.rename(columns={
    'PRODUCTCODE':'product_code',
    'PRODUCTLINE':'product_line',
    'MSRP':'msrp'
})
print("Products rows:", products.shape[0])
products.head()


Products rows: 109


,product_code,product_line,msrp
0,S10_1678,Motorcycles,95
1,S10_1949,Classic Cars,214
2,S10_2016,Motorcycles,118
3,S10_4698,Motorcycles,193
4,S10_4757,Classic Cars,136


In [ ]:
# Orders table
# The CSV has ORDERNUMBER as order id; ORDERDATE_parsed is parsed date
order_cols = [c for c in ['ORDERNUMBER','ORDERDATE_parsed','STATUS','QTR_ID','MONTH_ID','YEAR_ID','DEALSIZE','CUSTOMERNAME'] if c in df.columns or c=='ORDERDATE_parsed']
orders = df[['ORDERNUMBER','ORDERDATE_parsed','STATUS','QTR_ID','MONTH_ID','YEAR_ID','DEALSIZE','CUSTOMERNAME']].drop_duplicates(subset=['ORDERNUMBER']).copy()
orders = orders.rename(columns={
    'ORDERNUMBER':'order_number',
    'ORDERDATE_parsed':'order_date',
    'STATUS':'status',
    'QTR_ID':'qtr_id',
    'MONTH_ID':'month_id',
    'YEAR_ID':'year_id',
    'DEALSIZE':'deal_size',
    'CUSTOMERNAME':'customer_name'   # will map to customers.customer_name
})
orders['order_number'] = orders['order_number'].astype(str)  # keep as string to be safe
print("Orders rows:", orders.shape[0])
orders.head()


Orders rows: 307


,order_number,order_date,status,qtr_id,month_id,year_id,deal_size,customer_name
0,10107,2003-02-24,Shipped,1,2,2003,Small,Land of Toys Inc.
1,10121,2003-05-07,Shipped,2,5,2003,Small,Reims Collectables
2,10134,2003-07-01,Shipped,3,7,2003,Medium,Lyon Souveniers
3,10145,2003-08-25,Shipped,3,8,2003,Medium,Toys4GrownUps.com
4,10159,2003-10-10,Shipped,4,10,2003,Medium,Corporate Gift Ideas Co.


In [ ]:
# OrderDetails (line items)
item_cols = [c for c in ['ORDERNUMBER','PRODUCTCODE','QUANTITYORDERED','PRICEEACH','ORDERLINENUMBER','SALES'] if c in df.columns]
orderitems = df[item_cols].copy().reset_index(drop=True)
orderitems = orderitems.rename(columns={
    'ORDERNUMBER':'order_number',
    'PRODUCTCODE':'product_code',
    'QUANTITYORDERED':'quantity_ordered',
    'PRICEEACH':'price_each',
    'ORDERLINENUMBER':'order_line_number',
    'SALES':'sales'
})

# ensure numeric conversions
orderitems['quantity_ordered'] = pd.to_numeric(orderitems['quantity_ordered'], errors='coerce').fillna(0).astype(int)
orderitems['price_each'] = pd.to_numeric(orderitems['price_each'], errors='coerce')
orderitems['sales'] = pd.to_numeric(orderitems['sales'], errors='coerce')
orderitems['order_number'] = orderitems['order_number'].astype(str)
print("Order items rows:", orderitems.shape[0])
orderitems.head()


Order items rows: 2823


,order_number,product_code,quantity_ordered,price_each,order_line_number,sales
0,10107,S10_1678,30,95.70,2,2871.00
1,10121,S10_1678,34,81.35,5,2765.90
2,10134,S10_1678,41,94.74,2,3884.34
3,10145,S10_1678,45,83.26,6,3746.70
4,10159,S10_1678,49,100.00,14,5205.27


## Load: Using SQLAlchemy

In [ ]:
# Database connection using SQLAlchemy
DB_USER = 'root'
DB_PASSWORD = ''
DB_HOST = '127.0.0.1'
DB_PORT = 3307
DB_NAME = 'sales_db'

engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}", echo=False)
print("Engine created. Ready to write.")


Engine created. Ready to write.


In [ ]:
# write customers to DB and read back customer_id mapping
# write customer rows and then read customer_id & customer_name for mapping.
customers.to_sql('Customers', engine, if_exists='append', index=False, method='multi')
print("Customers written. Counting rows in DB...")
import pandas as pd
customers_db = pd.read_sql("SELECT customer_id, customer_name FROM Customers", engine)
print("Customers in DB:", len(customers_db))
customers_db.head()


Customers written. Counting rows in DB...
Customers in DB: 92


,customer_id,customer_name
0,1,Land of Toys Inc.
1,2,Reims Collectables
2,3,Lyon Souveniers
3,4,Toys4GrownUps.com
4,5,Corporate Gift Ideas Co.


In [ ]:
# write products and read back mapping table
products.to_sql('Products', engine, if_exists='append', index=False, method='multi')
products_db = pd.read_sql("SELECT product_id, product_code FROM Products", engine)
print("Products in DB:", len(products_db))
products_db.head()


Products in DB: 109


,product_id,product_code
0,1,S10_1678
1,2,S10_1949
2,3,S10_2016
3,4,S10_4698
4,5,S10_4757


In [ ]:
# map customer_name in orders to customer_id using customers_db
orders = orders.merge(customers_db, left_on='customer_name', right_on='customer_name', how='left')

# rename the mapping column to customer_id (DB expects this)
orders = orders.rename(columns={'customer_id':'customer_id'})
print("Orders after mapping customer_id (null = not found):")
display(orders.head())
print("Missing customer_id count:", orders['customer_id'].isnull().sum())


Orders after mapping customer_id (null = not found):


,order_number,order_date,status,qtr_id,month_id,year_id,deal_size,customer_name,customer_id
0,10107,2003-02-24,Shipped,1,2,2003,Small,Land of Toys Inc.,1
1,10121,2003-05-07,Shipped,2,5,2003,Small,Reims Collectables,2
2,10134,2003-07-01,Shipped,3,7,2003,Medium,Lyon Souveniers,3
3,10145,2003-08-25,Shipped,3,8,2003,Medium,Toys4GrownUps.com,4
4,10159,2003-10-10,Shipped,4,10,2003,Medium,Corporate Gift Ideas Co.,5


Missing customer_id count: 0


In [ ]:
# map product_code to product_id for orderitems
orderitems = orderitems.merge(products_db, left_on='product_code', right_on='product_code', how='left')
print("Order items after mapping product_id (null = not found):")
display(orderitems.head())
print("Missing product_id count:", orderitems['product_id'].isnull().sum())


Order items after mapping product_id (null = not found):


,order_number,product_code,quantity_ordered,price_each,order_line_number,sales,product_id
0,10107,S10_1678,30,95.70,2,2871.00,1
1,10121,S10_1678,34,81.35,5,2765.90,1
2,10134,S10_1678,41,94.74,2,3884.34,1
3,10145,S10_1678,45,83.26,6,3746.70,1
4,10159,S10_1678,49,100.00,14,5205.27,1


Missing product_id count: 0


In [ ]:
# prepare orders to insert and write to DB
orders_to_insert = orders[['order_number','order_date','status','qtr_id','month_id','year_id','deal_size','customer_id']].drop_duplicates(subset=['order_number'])
orders_to_insert.to_sql('Orders', engine, if_exists='append', index=False, method='multi')
print("Inserted Orders:", len(orders_to_insert))


Inserted Orders: 307


In [ ]:
# prepare order details and insert
orderdetails_to_insert = orderitems[['order_number','product_id','quantity_ordered','price_each','order_line_number','sales']].copy()
# drop rows with missing product_id or order_number to avoid FK errors
before = len(orderdetails_to_insert)
orderdetails_to_insert = orderdetails_to_insert.dropna(subset=['order_number','product_id'])
after = len(orderdetails_to_insert)
print(f"Dropping {before-after} order items missing order_number or product_id before insert")
orderdetails_to_insert.to_sql('OrderDetails', engine, if_exists='append', index=False, method='multi')
print("Inserted OrderDetails:", len(orderdetails_to_insert))


Dropping 0 order items missing order_number or product_id before insert
Inserted OrderDetails: 2823


## SQL Queries

In [ ]:
# quick verification: counts from DB
print("Total customers:", pd.read_sql("SELECT COUNT(*) AS cnt FROM Customers", engine))
print("Total products:", pd.read_sql("SELECT COUNT(*) AS cnt FROM Products", engine))
print("Total orders:", pd.read_sql("SELECT COUNT(*) AS cnt FROM Orders", engine))
print("Total order details:", pd.read_sql("SELECT COUNT(*) AS cnt FROM OrderDetails", engine))


Total customers:    cnt
0   92
Total products:    cnt
0  109
Total orders:    cnt
0  307
Total order details:     cnt
0  2823


In [ ]:
# show orders with total items and total sales aggregated from OrderDetails
q = """
SELECT o.order_number, o.order_date, c.customer_name, 
       SUM(d.quantity_ordered) AS total_items,
       SUM(d.sales) AS total_sales
FROM Orders o
LEFT JOIN OrderDetails d ON o.order_number = d.order_number
LEFT JOIN Customers c ON o.customer_id = c.customer_id
GROUP BY o.order_number, o.order_date, c.customer_name
ORDER BY o.order_date DESC
LIMIT 10;
"""
display(pd.read_sql(q, engine))


,order_number,order_date,customer_name,total_items,total_sales
0,10425,2005-05-31,La Rochelle Gifts,427.0,43784.69
1,10424,2005-05-31,Euro Shopping Channel,269.0,35133.34
2,10422,2005-05-30,Diecast Classics Inc.,76.0,6166.80
3,10423,2005-05-30,Petit Auto,111.0,8411.95
4,10421,2005-05-29,Mini Gifts Distributors Ltd.,75.0,7261.75
5,10420,2005-05-29,Souveniers And Things Co.,532.0,43971.43
6,10419,2005-05-17,Salzburg Collectables,509.0,59475.10
7,10417,2005-05-13,Euro Shopping Channel,259.0,31821.90
8,10416,2005-05-10,L'ordine Souveniers,460.0,41509.94
9,10415,2005-05-09,"Australian Collectables, Ltd",164.0,14378.09


In [ ]:
# top-selling products by quantity
q = """
SELECT p.product_code, p.product_line, SUM(d.quantity_ordered) AS qty_sold
FROM OrderDetails d
JOIN Products p ON d.product_id = p.product_id
GROUP BY p.product_code, p.product_line
ORDER BY qty_sold DESC
LIMIT 10;
"""
display(pd.read_sql(q, engine))


,product_code,product_line,qty_sold
0,S18_3232,Classic Cars,1774.0
1,S24_3856,Classic Cars,1052.0
2,S18_4600,Trucks and Buses,1031.0
3,S700_4002,Planes,1029.0
4,S12_4473,Trucks and Buses,1024.0
5,S24_3949,Planes,1008.0
6,S50_1341,Vintage Cars,999.0
7,S18_1097,Trucks and Buses,999.0
8,S18_2432,Trucks and Buses,998.0
9,S18_1342,Vintage Cars,997.0


In [ ]:
# Export snapshots to CSV
pd.read_sql("SELECT * FROM Orders LIMIT 100", engine).to_csv("orders_snapshot.csv", index=False)
pd.read_sql("SELECT * FROM Customers LIMIT 100", engine).to_csv("customers_snapshot.csv", index=False)
print("Snapshots saved: orders_snapshot.csv, customers_snapshot.csv")


Snapshots saved: orders_snapshot.csv, customers_snapshot.csv
